In [12]:
!git clone https://github.com/oreilly-japan/deep-learning-from-scratch.git

Cloning into 'deep-learning-from-scratch'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 453 (delta 1), reused 5 (delta 1), pack-reused 446
Receiving objects: 100% (453/453), 5.52 MiB | 9.73 MiB/s, done.
Resolving deltas: 100% (233/233), done.


In [13]:
!pip install pillow
import numpy as np
import matplotlib.pyplot as plt
import pickle

スライド４枚目：Softmax関数の実装

In [14]:
def softmax(x):
  x -= np.max(x,axis=-1,keepdims=True)
  return np.exp(x)/np.sum(np.exp(x),axis=-1,keepdims=True)

#'-1'の意味
#'keepdims'の意味

In [15]:
print(softmax([0,1,2,3]))

[0.0320586  0.08714432 0.23688282 0.64391426]


In [17]:
arr = np.arange(12).reshape(3,4)
print(arr)
print(softmax(arr))

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[0.0320586  0.08714432 0.23688282 0.64391426]
 [0.0320586  0.08714432 0.23688282 0.64391426]
 [0.0320586  0.08714432 0.23688282 0.64391426]]


スライド７枚目：mnistデータセットの読み込み

In [18]:
from deep_learning_scratch.dataset import mnist
from PIL import Image

In [21]:
(x_train,t_train),(x_test,t_test) = mnist.load_mnist(normalize=False,flatten=True,one_hot_label = False)
#形状を見てみる
for data in (x_train,t_train, x_test,t_test):
  print(data.shape)
 


(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [26]:
pil_img = Image.fromarray(x_train[0].reshape(28,28))
pil_img.show()


In [27]:
print(t_train[0])


5


スライド１０枚目：３層ニューラルネットワークの推論過程

In [30]:
#学習済みパラメータをロード（前と同じ）
def init_network():
  with open('/content/deep_learning_scratch/ch03/sample_weight.pkl','rb') as f:
    network = pickle.load(f)

  return network  

In [31]:
def sigmoid(x):
  #復習
  return 1/(1+np.exp(-x))

In [32]:
#３層のpredict関数（前と同じ）
def predict(x):
  #パラメータを取得
  network = init_network()
  w1,w2,w3 = network['W1'],network['W2'],network['W3']
  b1,b2,b3 = network['b1'],network['b2'],network['b3']

  a1 = np.dot(x,w1)+b1
  z1 = sigmoid(a1)
  a2 = np.dot(z1,w2)+b2
  z2 = sigmoid(a2)
  a3 = np.dot(z2,w3)+b3
  y = softmax(a3)
  return y

In [33]:
#mnistで推論＆可視化
train_size = x_train.shape[0]
batch_size = 100
batch_mask = np.random.choice(train_size,batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

#推論(100枚を一度に行う)
y_batch = predict(x_batch)

#精度はどのくらい？
y = np.argmax(y_batch,axis=1)
acc = np.sum(y==t_batch) / batch_size
print(acc)

0.94


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


スライド１５、１８枚目：クロスエントロピー誤差の実装

In [34]:
def cross_entropy_error(y,t):
  #1次元の場合（データが１つの場合）だけ２次元に直す
  if y.ndim == 1:
    y = y.reshape(1,-1)
    t = t.reshape(1,-1)

  #tはyと同じ形状のone_hotベクトル
  batch_size = y.shape[0]
  return -np.sum(t*np.log(y+1e-7))/ batch_size


In [35]:
print(y_batch.shape)

#正解ラベルをone-hotベクトル化
t_batch_onehot = np.zeros(y_batch.shape)
t_batch_onehot[np.arange(y_batch.shape[0]),t_batch] = 1

print(t_batch_onehot.shape)
print(cross_entropy_error(y_batch,t_batch_onehot))

(100, 10)
(100, 10)
0.24012603656083228


勾配の計算

In [39]:
def numerical_gradient_val(f,x,h = 1e-4):
  return (f(x+h) - f(x-h))/ (2*h)
def numerical_gradient_val2(f,x,h = 1e-4):
  return (f(x+h) - f(x))/ h



In [40]:
#適当な関数を設定
def circle(x,y):
  return x ** 2 + y ** 2


In [41]:
#(3,4)における勾配を計算
#xで偏微分
circle_x = lambda x:circle(x,4)
dx = numerical_gradient_val(circle_x,3)
dx2 = numerical_gradient_val2(circle_x,3)

#yで偏微分
circle_y = lambda y:circle(3,y)
dy = numerical_gradient_val(circle_y,4)
dy2 = numerical_gradient_val(circle_y,4)

print((dx,dy))
print((dx2,dy2))
print((2*3,2*4))
#丸め誤差のお話
#どうしてf(x+h)-f(h)にしないのか問題

(6.00000000000378, 7.999999999999119)
(6.000099999994291, 7.999999999999119)
(6, 8)


In [42]:
#勾配降下法のめちゃくちゃ単純なやつ
lr = 0.1
iters = 10000

#初期値
(x,y) = (10,10)

#学習
for i in range(iters):
  #xで偏微分
  circle_x = lambda x:circle(x,y)
  dx = numerical_gradient_val(circle_x,x)

  #yで偏微分
  circle_y = lambda y:circle(x,y)
  dy = numerical_gradient_val(circle_y,y)

  #パラメータの更新
  x -= lr*dx
  y -= lr*dy

print(x,y)


6.6836193494284636e-21 6.6836193494284636e-21
